# **1. Install Necessary Libraries**

In [ ]:
pip install --upgrade transformers accelerate

In [ ]:
!pip install datasets -q
!pip install rouge-score -q
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
import os
os.kill(os.getpid(), 9)

# **2. Import Libraries and Set Up Device**

In [ ]:
import torch
from transformers import EncoderDecoderModel, BertTokenizer

# from datasets import load_dataset, load_metric
import datasets

# Import necessary libraries
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import numpy as np
import nltk
import evaluate
from rouge_score import rouge_scorer
import os

# Download the NLTK Punkt tokenizer for sentence splitting
nltk.download('punkt')

# Set up the device for computation (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

import numpy as np

Using device: cuda


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Import Files from Google Drive to Colab
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# **3. Load and Prepare the Dataset**


In [ ]:
from datasets import load_from_disk, load_dataset

# dataset = load_from_disk("/content/drive/MyDrive/WACL4_COLING_2025/portion_dataset/concatenate_only_trains_parts")
dataset = load_dataset("azzedine/Goud-sum_v2")


dataset

In [ ]:
# Preprocessing function
# max_input_length = 350
# max_target_length = 50

encoder_max_length =  256
decoder_max_length =  32

max_input_length = encoder_max_length
max_target_length = decoder_max_length
def preprocess_function(examples):
    inputs = examples['article']
    targets = examples['headline']
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding='max_length')

    # Adjust labels for BERT2BRT
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_seq]
        for label_seq in labels["input_ids"]
        ]


    model_inputs['labels'] = labels['input_ids']
    return model_inputs



# **6. Load BERT2BERT Model and Tokenizer**

In [ ]:
from transformers import BertModel
from transformers import BertTokenizer, EncoderDecoderModel
import torch


model_name = 'aubmindlab/bert-base-arabertv02-twitter'



# Detect the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the EncoderDecoderModel with BERT as both encoder and decoder
# model = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name).to(device)

# tokenizer = AutoTokenizer.from_pretrained(model_name)
 # model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer = BertTokenizer.from_pretrained(model_name)
model = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name)


model = model.to(device)
# Set special tokens
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Configure generation parameters
model.config.vocab_size = model.config.decoder.vocab_size
model.config.max_length = max_target_length
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.num_beams = 5  # Correctly set as an integer
model.config.length_penalty = 2.0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/751k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertLMHeadModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.cros

# **7. Tokenize the Datasets**

In [ ]:
# Apply the preprocessing function to all splits (train, validation, test)
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=['article', 'headline', 'categories'],  # Remove original columns after tokenization
)
tokenized_datasets

In [ ]:
import evaluate
from rouge_score import rouge_scorer

# Initialize ROUGE
rouge = evaluate.load('rouge')
r_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'],  tokenizer=tokenizer)


def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)


    candidate_summaries = pred_str
    reference_summaries = label_str
    print(f"candidate_summaries = {candidate_summaries}")
    print(f"reference_summaries = {reference_summaries}")
    # Evaluate ROUGE scores

    # Store scores
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    # Calculate scores for the list of texts
    for ref, gen in zip(reference_summaries, candidate_summaries):
      score = r_scorer.score(gen, ref)
      # Append scores to respective lists
      scores['rouge1'].append(score['rouge1'].fmeasure)
      scores['rouge2'].append(score['rouge2'].fmeasure)
      scores['rougeL'].append(score['rougeL'].fmeasure)

    # Calculate mean scores
    mean_scores = {metric: np.mean(values) for metric, values in scores.items()}


    # Calculate the average length of the reference summaries
    reference_lens = [len(label.split()) for label in candidate_summaries]
    mean_scores['ref_len'] = np.mean(reference_lens)

    # Calculate the average length of the generated summaries
    prediction_lens = [len(pred.split()) for pred in reference_summaries]
    mean_scores['gen_len'] = np.mean(prediction_lens)
    print(f"scores = {mean_scores}")
    return mean_scores

# **9. Set Up Training Arguments**

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

train_batch_size = 20
val_batch_size = 4

# Configure the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/WACL4_COLING_2025/comparaison_with_papers/models/AraBERT/checkpoints",    # Output directory
    eval_strategy="epoch",             # Evaluate every epoch
    save_strategy="epoch",                   # Save checkpoint every epoch
    learning_rate=2e-5,                      # Learning rate
    per_device_train_batch_size=train_batch_size,           # Training batch size per device
    per_device_eval_batch_size=val_batch_size,            # Evaluation batch size per device
    weight_decay=0.01,                       # Weight decay for regularization
    gradient_accumulation_steps=8,           # Gradient accumulation steps
    num_train_epochs=20,                      # Number of training epochs
    predict_with_generate=True,              # Use generate() for evaluation
    logging_dir='/content/drive/MyDrive/WACL4_COLING_2025/models/AraBERT/bert2bert_logs',
    fp16=torch.cuda.is_available(),          # Use mixed precision if supported
    save_total_limit=None,                      # Keep only the last 3 checkpoints
    load_best_model_at_end=True,             # Load the best model at the end
    metric_for_best_model='eval_rouge1',     # Metric to select the best model
    greater_is_better=True                  # Higher metric is better

)


# **10. Initialize the Trainer**

In [ ]:
tokenized_datasets['validation']

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 9497
})

In [ ]:
# Initialize the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,                             # The instantiated 🤗 Transformers model to be trained
    args=training_args,                      # Training arguments
    train_dataset=tokenized_datasets['train'],    # Training dataset ['train']
    eval_dataset=tokenized_datasets['validation'],# Evaluation dataset ['validation']
    tokenizer=tokenizer,                     # Tokenizer
    compute_metrics=compute_metrics,         # Function to compute metrics
)

# **11. Train the Model**

In [ ]:
# Start training
trainer.train()

In [ ]:
# After training is complete
best_model_dir = '/content/drive/MyDrive/WACL4_COLING_2025/comparaison_with_papers/models/best_model'  # Specify your desired directory
trainer.save_model(best_model_dir)
tokenizer.save_pretrained(best_model_dir)

('/content/drive/MyDrive/WACL4_COLING_2025/comparaison_with_papers/models/best_model/tokenizer_config.json',
 '/content/drive/MyDrive/WACL4_COLING_2025/comparaison_with_papers/models/best_model/special_tokens_map.json',
 '/content/drive/MyDrive/WACL4_COLING_2025/comparaison_with_papers/models/best_model/vocab.txt',
 '/content/drive/MyDrive/WACL4_COLING_2025/comparaison_with_papers/models/best_model/added_tokens.json')

**Evaluate on Test set**

# **13. Generate Summaries and Save Results**

In [ ]:
best_checkpoint = trainer.state.best_model_checkpoint
print(f"The best model checkpoint is saved at: {best_checkpoint}")

The best model checkpoint is saved at: /content/drive/MyDrive/WACL4_COLING_2025/comparaison_with_papers/models/checkpoints/checkpoint-29180


In [ ]:
best_checkpoint

'/content/drive/MyDrive/WACL4_COLING_2025/comparaison_with_papers/models/checkpoints/checkpoint-29180'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(best_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(best_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
tokenized_datasets['test']

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 9497
})

In [ ]:
# Generate Summaries Function
def generate_summary(example):
    # Tokenize the input text
    inputs = tokenizer(
        example['article'],
        max_length=max_input_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    # Move tensors to the computation device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate the summary using the model
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_target_length,
        num_beams=5,           # Number of beams for beam search
        early_stopping=True    # Stop when at least num_beams sentences are finished
    )

    # Decode the generated summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Add the generated summary to the example with a model-specific flag
    example["generated_summary_BERT2BERT"] = summary
    return example


In [ ]:
# Apply the generate_summary function to each example in the test set
generate_dataset = dataset['test'].map(generate_summary)

Map:   0%|          | 0/9497 [00:00<?, ? examples/s]

# **14. Save Generated Summaries**

In [ ]:
# Save the dataset with generated summaries to disk
generate_dataset.save_to_disk('/content/drive/MyDrive/WACL4_COLING_2025//AraBERT/model_result')


Saving the dataset (0/1 shards):   0%|          | 0/9497 [00:00<?, ? examples/s]